In [19]:
import os
import shutil
import subprocess
import shlex
import numpy as np
import rasterio as rio
from rasterio.warp import reproject, Resampling
from pyproj import CRS
from pyproj.crs import CompoundCRS

In [ ]:
os.getcwd()

In [ ]:
# Load the input CSV file
input_file = "ohio1.csv"  # Change this to your CSV file path
output_file = "ohio_out.csv"

# Read the CSV file
df = pd.read_csv(input_file, names=["latitude", "longitude", "height"])

# Define the CRS for NAD27 and NAVD88 (horizontal and vertical datums)
crs_nad27 = CRS.from_epsg(4267)  # EPSG:4267 = NAD27 (horizontal)
crs_navd88 = CRS.from_epsg(6319)  # EPSG:6319 = NAVD88 (vertical)

# Define the target CRS for EGM2008 geoid model (vertical)
crs_egm2008 = CRS.from_epsg(3855)  # EPSG:3855 = EGM2008 (vertical)

# Initialize the transformer for converting NAD27 (horizontal) to WGS84 (for modern transformations)
transformer_horiz = Transformer.from_crs(crs_nad27, CRS.from_epsg(4326), always_xy=True)  # NAD27 to WGS84

# Initialize the vertical transformation for NAVD88 to EGM2008
transformer_vert = Transformer.from_crs(crs_navd88, crs_egm2008, always_xy=True)  # NAVD88 to EGM2008

# Function to transform height with horizontal and vertical adjustments
def transform_coordinates(lat, lon, height):
    if pd.isna(lat) or pd.isna(lon) or pd.isna(height):
        return None  # Skip if any value is NaN
    try:
        # First, transform the horizontal coordinates from NAD27 to WGS84 (EPSG:4326)
        lon_wgs84, lat_wgs84 = transformer_horiz.transform(lon, lat)
        
        # Then, transform the height from NAVD88 to EGM2008 based on the corrected latitude and longitude
        height_egm2008 = transformer_vert.transform(lon_wgs84, lat_wgs84, height)
        return height_egm2008
    except Exception as e:
        return None  # Return None if the transformation fails

# Apply the transformation row by row
height_egm2008 = []
for _, row in df.iterrows():
    transformed_height = transform_coordinates(row["latitude"], row["longitude"], row["height"])
    height_egm2008.append(transformed_height)

# Add the new column to the DataFrame
df["height_egm2008"] = height_egm2008



In [ ]:
# Save the result to a new CSV file
df.to_csv(output_file, index=False)
print(f"Transformation complete. Output saved to {output_file}")

# France

In [ ]:
# ── YOUR PATHS (as provided) ───────────────────────────────────────────────────
IN_DEM   = r"/Users/daniel/Library/CloudStorage/GoogleDrive-ejdvc757@gmail.com/Other computers/My MacBook Pro/PhD/Dissertation/0_data/External/Bathymetries/Europa/Garonne/Garonne_R.tif"
EGM_GRID = r"/Users/daniel/Library/CloudStorage/GoogleDrive-ejdvc757@gmail.com/Other computers/My MacBook Pro/PhD/Dissertation/0_data/External/Geoids/us_nga_egm2008_25.tif"
OUT_DEM  = r"/Users/daniel/Library/CloudStorage/GoogleDrive-ejdvc757@gmail.com/Other computers/My MacBook Pro/PhD/Dissertation/0_data/External/Bathymetries/Europa/Garonne/Garonne_egm2008.tif"
TMP_OUT  = r"/Users/daniel/Downloads/_tmp_Garonne_egm2008.tif"
GDAL_EDIT = "gdal_edit.py"  # set full path if not on PATH, e.g. "/opt/homebrew/bin/gdal_edit.py"
# ────────────────────────────────────────────────────────────────────────────────

# 1) Load DEM
with rasterio.open(IN_DEM) as src_dem:
    dem = src_dem.read(1).astype("float32")
    dem_transform = src_dem.transform
    dem_crs = src_dem.crs
    dem_nodata = src_dem.nodata
    Hgt, Wdt = src_dem.height, src_dem.width

if dem_nodata is None:
    dem_nodata = np.float32(-9999.0)
dem = np.where(np.isnan(dem), dem_nodata, dem)

# Assume Lambert-93 if CRS missing
if dem_crs is None:
    dem_crs = CRS.from_epsg(2154)

# 2) Load EGM2008 grid and resample to DEM grid
with rasterio.open(EGM_GRID) as src_geoid:
    geoid_resampled = np.empty((Hgt, Wdt), dtype="float32")
    reproject(
        source=rasterio.band(src_geoid, 1),
        destination=geoid_resampled,
        src_transform=src_geoid.transform,
        src_crs=src_geoid.crs,
        src_nodata=src_geoid.nodata,
        dst_transform=dem_transform,
        dst_crs=dem_crs,
        dst_nodata=np.nan,
        resampling=Resampling.bilinear,
    )

# 3) Compute orthometric heights: H = h - N
mask = (dem == dem_nodata) | np.isnan(geoid_resampled)
H = dem - geoid_resampled
H = np.where(mask, dem_nodata, H).astype("float32")

# 4) Prepare output profile (no tiling → avoids TIFF tile issues on cloud drives)
out_profile = {
    "driver": "GTiff",
    "width": Wdt,
    "height": Hgt,
    "count": 1,
    "dtype": "float32",
    "nodata": dem_nodata,
    "transform": dem_transform,
    "crs": dem_crs,   # upgraded to compound CRS below if possible
    "compress": "lzw",
    "tiled": False,
    "BIGTIFF": "IF_SAFER",
}

# Try to set compound CRS (Lambert-93 + EGM2008 height)
compound_set = False
try:
    h_crs = CRS.from_epsg(2154)
    v_crs = CRS.from_epsg(3855)
    compound = CompoundCRS(name="Lambert-93 + EGM2008 height", components=[h_crs, v_crs])
    out_profile["crs"] = compound.to_wkt("WKT2_2019")
    compound_set = True
except Exception as e:
    out_profile["crs"] = h_crs  # keep horizontal at least
    print("⚠️ Could not set compound CRS in-writer:", e)

# 5) Write to LOCAL TEMP (delete if exists), then move
tmp_dir = os.path.dirname(TMP_OUT)
os.makedirs(tmp_dir, exist_ok=True)
if os.path.exists(TMP_OUT):
    try:
        os.remove(TMP_OUT)
    except Exception:
        base, ext = os.path.splitext(TMP_OUT)
        TMP_OUT = f"{base}_new{ext}"

with rasterio.open(TMP_OUT, "w", **out_profile) as dst:
    dst.write(H, 1)

# If compound CRS not set, try tagging with gdal_edit.py
if not compound_set:
    try:
        cmd = f'{GDAL_EDIT} -a_srs "EPSG:2154+3855" "{TMP_OUT}"'
        subprocess.run(shlex.split(cmd), check=True)
        print("✅ Tagged vertical CRS via gdal_edit.py → EPSG:2154+3855")
    except Exception as e:
        print("⚠️ gdal_edit.py tagging failed; numeric heights are correct but metadata may lack vertical CRS.", e)

# Ensure target folder exists; if an old/broken TIFF exists there, remove it first
out_dir = os.path.dirname(OUT_DEM)
os.makedirs(out_dir, exist_ok=True)
if os.path.exists(OUT_DEM):
    try:
        os.remove(OUT_DEM)
    except Exception:
        base, ext = os.path.splitext(OUT_DEM)
        OUT_DEM = f"{base}_v2{ext}"
        print("ℹ️ Could not remove existing output; writing new file:", OUT_DEM)

# Move temp → final
shutil.move(TMP_OUT, OUT_DEM)

# 6) Quick stats & a tiny diagnostic
valid = ~mask
print("Done ✅  Wrote:", OUT_DEM)
if np.any(valid):
    arr = H[valid]
    q = np.nanpercentile(arr, [0, 5, 50, 95, 100])
    print(f"Orthometric H stats (valid px): min={q[0]:.3f}, p5={q[1]:.3f}, median={q[2]:.3f}, p95={q[3]:.3f}, max={q[4]:.3f}")
    # Diagnostic: mean difference vs original and correlation with -N
    dif = (H - dem)[valid]
    mean_diff = float(np.nanmean(dif))
    std_diff  = float(np.nanstd(dif))
    # correlation with -N
    Nv = geoid_resampled[valid]
    a = dif - np.nanmean(dif); b = (-Nv) - np.nanmean(-Nv)
    denom = np.sqrt(np.nanmean(a*a) * np.nanmean(b*b))
    corr_mN = float(np.nanmean(a*b)/denom) if denom > 0 else np.nan
    print(f"Δ(H - original dem): mean={mean_diff:.3f} std={std_diff:.3f} | Corr(Δ , -N)={corr_mN:.3f}")
else:
    print("Warning: no valid pixels after masking.")

Done ✅  Wrote: /Users/daniel/Library/CloudStorage/GoogleDrive-ejdvc757@gmail.com/Other computers/My MacBook Pro/PhD/Dissertation/0_data/External/Bathymetries/Europa/Garonne/Garonne_egm2008.tif
Orthometric H stats (valid px): min=-52.899, p5=-47.910, median=-47.570, p95=-19.687, max=46.241
Δ(H - original dem): mean=-47.663 std=0.188 | Corr(Δ , -N)=1.000
